In [7]:
import requests
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

In [8]:
def get_mails(line):
    pattern = r'[\w.+-]+@[\w-]+\.[\w.-]+'
    emails = re.findall(pattern, line)
    return emails

def get_phone_numbers(line):
    pattern = r'\b\+?[7,8]([\s, \-,\(]*?\d{3}[\s, \-,\)]*?\d{3}[\s*,\-]?\d{2}[\s*,\-,\(]?\d{2})\b'
    phones = re.findall(pattern, line)
    return phones
        


In [9]:
urls = pd.read_csv('filter_domains.csv', header=None)
urls = urls[0]

In [17]:
def parse_one_url(url):
    req = requests.get(url)
    soup = bs(req.text, 'html.parser')
    text = soup.get_text()
    phones = get_phone_numbers(text)
    mails = get_mails(text)
    try:
        req = requests.get(url + '/contact') 
        soup = bs(req.text, 'html.parser')
        text = soup.get_text()
        phones += get_phone_numbers(text)
        mails += get_mails(text)
    except:
       pass
    try:
        req = requests.get(url + '/contacts') 
        soup = bs(req.text, 'html.parser')
        text = soup.get_text()
        phones += get_phone_numbers(text)
        mails += get_mails(text)
    except:
       pass
      

    a = [url, list(set(phones)), list(set(mails))];
    return a

In [11]:
result = []
cnt = 0

In [41]:

for i in urls[cnt + 5:]:
    result.append(parse_one_url(i))
    cnt += 1
    if cnt % 5 == 0:
        print(cnt)

3855


KeyboardInterrupt: ignored

In [42]:
len(result)

3858

In [43]:
df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)

<ipython-input-43-2ff9ff6bc1c6>:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)


In [44]:
df.to_csv('tel_and_mail.csv', index=False)